In [389]:
import os, glob, time, pandas as pd
from datetime import datetime

In [390]:
def get_file_last_time_of_change(fname):
    try:
        mtime = os.path.getmtime(fname)
    except OSError:
        mtime = 0
    return mtime

In [391]:
def process_changed_files(list_of_changed_files):
    for fname in list_of_changed_files:
        if '0~' in fname:
            update_trade(fname)
        elif '2~' in fname:
            update_current(fname)

In [392]:
################# UPDATING ##################

In [393]:
main_batch_max_len = 300   # 5 min
main_batch = pd.DataFrame({'Timestamp': 0, 'curr_BTC_USD': 0, 'curr_ETH_USD': 0,
                           'curr_LTC_USD': 0, 'curr_ETH_BTC': 0, 'curr_LTC_BTC': 0,
                           'tr_BTC_USD_buy': 0, 'tr_ETH_USD_buy': 0, 'tr_LTC_USD_buy': 0,
                           'tr_ETH_BTC_buy': 0, 'tr_LTC_BTC_buy': 0, 'tr_BTC_USD_sell': 0,
                           'tr_ETH_USD_sell': 0, 'tr_LTC_USD_sell': 0, 'tr_ETH_BTC_sell': 0,
                           'tr_LTC_BTC_sell': 0}, index = [0])
fname_dict = {'2~Poloniex~BTC~USD.dat': 'curr_BTC_USD', '2~Poloniex~ETH~BTC.dat': 'curr_ETH_BTC',
              '2~Poloniex~ETH~USD.dat': 'curr_ETH_USD', '2~Poloniex~LTC~BTC.dat': 'curr_LTC_BTC',
              '2~Poloniex~LTC~USD.dat': 'curr_LTC_USD'}

In [394]:
def load_trade(fname):
    return pd.read_csv(fname, sep=";", header = None, names = ['Timestamp', 'Flag', 'Total'], parse_dates=['Timestamp'], index_col=['Timestamp'])

In [395]:
def load_current(fname):
    return pd.read_csv(fname, sep=";", header = None, names = ['Timestamp', 'Value'], parse_dates=['Timestamp'], index_col=['Timestamp'])

In [396]:
def update_trade(fname):
    print(time.asctime(time.localtime(time.time())), 'Updating trade:', fname)
    #df_new_trade = time_repair_trade(load_trade(fname))

In [397]:
def update_current(fname):
    print(time.asctime(time.localtime(time.time())), 'Updating current:', fname)
    df_new_current = load_current(fname).resample('S').mean().interpolate()
    print(df_new_current)

In [398]:
################# MAIN LOOP ##################

In [399]:
last_changed_times = []
last_changed_times_tmp = []
list_of_changed_files = []
last_time = 0

time_divider = 60 #60*10 ... ten minutes

last_time = -1

while True: #for i in range(120):     #while True:
    now_time = str(round(time.time()/time_divider))
    time_changed = False
    if last_time != now_time:
        last_changed_times_tmp = []
        time_changed = True
        last_time = now_time
        files_name = glob.glob('.\\data\\' + last_time + '\\*.dat')
        for fname in files_name:
            last_change_time_file = get_file_last_time_of_change(fname)
            last_changed_times_tmp.append(last_change_time_file)
        
    if not time_changed:
        files_name = glob.glob('.\\data\\' + now_time + '\\*.dat')

    for fname in files_name:
        last_changed_times.append(get_file_last_time_of_change(fname))

    if len(last_changed_times) > len(last_changed_times_tmp):
        last_changed_times_tmp = last_changed_times
        
    for j in range(len(files_name)):
        if last_changed_times_tmp[j] != last_changed_times[j]:
            list_of_changed_files.append(files_name[j])
    last_changed_times_tmp = last_changed_times
    last_changed_times = []
    
    process_changed_files(list_of_changed_files)
    list_of_changed_files = []
    
    time.sleep(1)   

Fri Jun 30 00:17:36 2017 Updating trade: .\data\24979578\0~Poloniex~BTC~USD.dat
Fri Jun 30 00:17:36 2017 Updating current: .\data\24979578\2~Poloniex~BTC~USD.dat
                           Value
Timestamp                       
2017-06-30 12:06:08  2498.299034
2017-06-30 12:06:09  2498.284978
2017-06-30 12:06:10  2498.270923
2017-06-30 12:06:11  2498.256868
2017-06-30 12:06:12  2498.242813
2017-06-30 12:06:13  2498.228758
2017-06-30 12:06:14  2498.214703
2017-06-30 12:06:15  2498.200648
2017-06-30 12:06:16  2498.186593
2017-06-30 12:06:17  2498.172538
2017-06-30 12:06:18  2498.158483
2017-06-30 12:06:19  2498.144427
2017-06-30 12:06:20  2498.197901
2017-06-30 12:06:21  2498.251374
2017-06-30 12:06:22  2498.304847
2017-06-30 12:06:23  2498.358321
2017-06-30 12:06:24  2498.411794
2017-06-30 12:06:25  2498.465267
2017-06-30 12:06:26  2498.518740
2017-06-30 12:06:27  2498.572214
2017-06-30 12:06:28  2498.625687
2017-06-30 12:06:29  2498.679160
2017-06-30 12:06:30  2498.732634
2017-06-30 12

Fri Jun 30 00:19:01 2017 Updating trade: .\data\24979579\0~Poloniex~ETH~BTC.dat
Fri Jun 30 00:19:01 2017 Updating current: .\data\24979579\2~Poloniex~ETH~BTC.dat
                        Value
Timestamp                    
2017-06-30 12:06:08  0.117400
2017-06-30 12:06:09  0.117400
2017-06-30 12:06:10  0.117388
2017-06-30 12:06:11  0.117377
2017-06-30 12:06:12  0.117365
2017-06-30 12:06:13  0.117353
2017-06-30 12:06:14  0.117348
2017-06-30 12:06:15  0.117342
2017-06-30 12:06:16  0.117336
2017-06-30 12:06:17  0.117330
2017-06-30 12:06:18  0.117350
2017-06-30 12:06:19  0.117342
2017-06-30 12:06:20  0.117333
2017-06-30 12:06:21  0.117325
2017-06-30 12:06:22  0.117317
2017-06-30 12:06:23  0.117308
2017-06-30 12:06:24  0.117300
2017-06-30 12:06:25  0.117300
2017-06-30 12:06:26  0.117310
2017-06-30 12:06:27  0.117320
2017-06-30 12:06:28  0.117330
2017-06-30 12:06:29  0.117340
2017-06-30 12:06:30  0.117350
2017-06-30 12:06:31  0.117360
2017-06-30 12:06:32  0.117370
2017-06-30 12:06:33  0.11738

KeyboardInterrupt: 

In [ ]:
df_current = load_current('.\\data\\24979507\\2~Poloniex~ETH~BTC.dat')
df_current.head()

In [ ]:
df_current = df_current.resample('S').mean().interpolate()
print(df_current_copy_rep)

In [ ]:
idx = pd.date_range(df_current_copy.index.min(), df_current_copy.index.max(), freq='S')
print('start:', block_start, '-- end:', block_end)
#print(idx)

In [ ]:
df_current_copy.index = pd.DatetimeIndex(df_current_copy.index)
df_current_copy.head()

In [ ]:
df_current_copy = df_current_copy.reindex(idx, fill_value=0)
df_current_copy.head()

In [ ]:
df_current_copy[df_current_copy.index.duplicated()]